In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd, time
import warnings, json, sys, requests, gzip, io, urllib, requests, os

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib.colors import LogNorm

from penquins import Kowalski
from sklearn.model_selection import train_test_split

from bson.json_util import loads, dumps
from astropy.io import fits
from astropy.io.fits.verify import VerifyWarning 
warnings.filterwarnings("ignore", category=VerifyWarning)

BOLD = "\033[1m"; END  = "\033[0m"

with open('misc/credentials.json', 'r') as f:
    creds = json.load(f)

In [62]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%config InlineBackend.figure_format = 'retina'

plt.rcParams.update({
    "font.family": "Times New Roman",
    "font.size": 12,
})
plt.rcParams['axes.linewidth'] = 1

k = Kowalski(username=creds['kowalski_username'], password=creds['kowalski_password'])
api_token = creds['fritz_api_key']
assert(k.ping())

external_HDD = "/Volumes/NRExternal3/trainv4 data/"
to_desktop = "/Users/nabeelr/Desktop/"


### Query the BTS Sample Explorer

In [3]:
# Old queries
# bts_trues_url      = "https://sites.astro.caltech.edu/ztf/bts/explorer.php?f=s&subsample=trans&classstring=&classexclude=&ps1img=y&lcfig=y&ztflink=lasair&lastdet=&startsavedate=&startpeakdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=&startabsmag=&starthostabs=&starthostcol=&startb=&startav=&endsavedate=&endpeakdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=18.5&endabsmag=&endhostabs=&endhostcol=&endb=&endav=&sort=peakmag&format=csv"
# bts_dim_falses_url = "https://sites.astro.caltech.edu/ztf/bts/explorer.php?f=s&subsample=trans&classstring=&classexclude=&quality=y&purity=y&ps1img=y&lcfig=y&ztflink=lasair&lastdet=&startsavedate=&startpeakdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=18.5&startabsmag=&starthostabs=&starthostcol=&startb=&startav=&endsavedate=&endpeakdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=&endabsmag=&endhostabs=&endhostcol=&endb=&endav=&sort=peakmag&reverse=y&format=csv"

# bts_trues_url      = "https://sites.astro.caltech.edu/ztf/bts/explorer.php?f=s&subsample=trans&classstring=&classexclude=&passok=y&refok=y&dateok=y&purity=y&ps1img=y&lcfig=y&ztflink=lasair&lastdet=&startsavedate=&startpeakdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=&startabsmag=&starthostabs=&starthostcol=&startb=&startav=&endsavedate=&endpeakdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=18.5&endabsmag=&endhostabs=&endhostcol=&endb=&endav=&sort=peakmag&format=csv"
# bts_var_falses_url = "https://sites.astro.caltech.edu/ztf/bts/explorer.php?f=s&subsample=var&classstring=&classexclude=&ztflink=lasair&lastdet=&startsavedate=&startpeakdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=&startabsmag=&starthostabs=&starthostcol=&startb=&startav=&endsavedate=&endpeakdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=&endabsmag=&endhostabs=&endhostcol=&endb=&endav=&format=csv"
# bts_dim_falses_url = "https://sites.astro.caltech.edu/ztf/bts/explorer.php?f=s&subsample=trans&classstring=&classexclude=&passok=y&refok=y&dateok=y&purity=y&ps1img=y&lcfig=y&ztflink=lasair&lastdet=&startsavedate=&startpeakdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=18.5&startabsmag=&starthostabs=&starthostcol=&startb=&startav=&endsavedate=&endpeakdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=&endabsmag=&endhostabs=&endhostcol=&endb=&endav=&sort=peakmag&reverse=y&format=csv"

# # v2
# query_urls = {
#     "rcf_trues":           "http://sites.astro.caltech.edu/ztf/rcf/explorer.php?f=s&coverage=any&samprcf=y&subsample=trans&classstring=&classexclude=&refok=y&purity=y&ps1img=y&lcfig=y&ztflink=fritz&startsavedate=&startpeakdate=&startlastdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=&startlastmag=&startabsmag=&starthostabs=&starthostcol=&startsavevis=&startlatevis=&startcurrvis=&startb=&startav=&endsavedate=&endpeakdate=&endlastdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=18.5&endlastmag=&endabsmag=&endhostabs=&endhostcol=&endsavevis=&endlatevis=&endcurrvis=&endb=&endav=&sort=peakmag&format=csv",
#     "rcf_dim_falses":      "http://sites.astro.caltech.edu/ztf/rcf/explorer.php?f=s&coverage=any&samprcf=y&subsample=trans&classstring=&classexclude=&covok=y&refok=y&purity=y&ps1img=y&lcfig=y&ztflink=fritz&startsavedate=&startpeakdate=&startlastdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=18.5&startlastmag=&startabsmag=&starthostabs=&starthostcol=&startsavevis=&startlatevis=&startcurrvis=&startb=&startav=&endsavedate=&endpeakdate=&endlastdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=&endlastmag=&endabsmag=&endhostabs=&endhostcol=&endsavevis=&endlatevis=&endcurrvis=&endb=&endav=&sort=peakmag&format=csv",
#     "rcf_var_falses":      "http://sites.astro.caltech.edu/ztf/rcf/explorer.php?f=s&coverage=any&samprcf=y&subsample=var&classstring=&classexclude=&refok=y&ps1img=y&lcfig=y&ztflink=fritz&startsavedate=&startpeakdate=&startlastdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=&startlastmag=&startabsmag=&starthostabs=&starthostcol=&startsavevis=&startlatevis=&startcurrvis=&startb=&startav=&endsavedate=&endpeakdate=&endlastdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=&endlastmag=&endabsmag=&endhostabs=&endhostcol=&endsavevis=&endlatevis=&endcurrvis=&endb=&endav=&sort=peakmag&format=csv",

#     "rcf_deep_trues"     : "http://sites.astro.caltech.edu/ztf/rcf/explorer.php?f=s&coverage=any&sampdeep=y&subsample=trans&classstring=&classexclude=&refok=y&purity=y&ps1img=y&lcfig=y&ztflink=fritz&startsavedate=&startpeakdate=&startlastdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=&startlastmag=&startabsmag=&starthostabs=&starthostcol=&startsavevis=&startlatevis=&startcurrvis=&startb=&startav=&endsavedate=&endpeakdate=&endlastdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=18.5&endlastmag=&endabsmag=&endhostabs=&endhostcol=&endsavevis=&endlatevis=&endcurrvis=&endb=&endav=&sort=peakmag&format=csv",
#     "rcf_deep_dim_falses": "http://sites.astro.caltech.edu/ztf/rcf/explorer.php?f=s&coverage=any&sampdeep=y&subsample=trans&classstring=&classexclude=&covok=y&refok=y&purity=y&ps1img=y&lcfig=y&ztflink=fritz&startsavedate=&startpeakdate=&startlastdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=18.5&startlastmag=&startabsmag=&starthostabs=&starthostcol=&startsavevis=&startlatevis=&startcurrvis=&startb=&startav=&endsavedate=&endpeakdate=&endlastdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=&endlastmag=&endabsmag=&endhostabs=&endhostcol=&endsavevis=&endlatevis=&endcurrvis=&endb=&endav=&sort=peakmag&format=csv",
#     "rcf_deep_var_falses": "http://sites.astro.caltech.edu/ztf/rcf/explorer.php?f=s&coverage=any&sampdeep=y&subsample=var&classstring=&classexclude=&refok=y&ps1img=y&lcfig=y&ztflink=fritz&startsavedate=&startpeakdate=&startlastdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=&startlastmag=&startabsmag=&starthostabs=&starthostcol=&startsavevis=&startlatevis=&startcurrvis=&startb=&startav=&endsavedate=&endpeakdate=&endlastdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=&endlastmag=&endabsmag=&endhostabs=&endhostcol=&endsavevis=&endlatevis=&endcurrvis=&endb=&endav=&sort=peakmag&format=csv"
# }

# # v3.1
# query_urls = {
#     "rcf_trues":    "http://sites.astro.caltech.edu/ztf/rcf/explorer.php?f=s&coverage=any&samprcf=y&sampdeep=y&subsample=trans&classstring=&classexclude=&refok=y&purity=y&ps1img=y&lcfig=y&ztflink=fritz&startsavedate=&startpeakdate=&startlastdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=&startlastmag=&startabsmag=&starthostabs=&starthostcol=&startsavevis=&startlatevis=&startcurrvis=&startb=&startav=&endsavedate=&endpeakdate=&endlastdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=18.5&endlastmag=&endabsmag=&endhostabs=&endhostcol=&endsavevis=&endlatevis=&endcurrvis=&endb=&endav=&sort=peakmag&format=csv",
#     "rcf_dim":      "http://sites.astro.caltech.edu/ztf/rcf/explorer.php?f=s&coverage=any&samprcf=y&subsample=all&classstring=&classexclude=&covok=y&refok=y&lcfig=y&ztflink=fritz&startsavedate=&startpeakdate=&startlastdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=18.5&startlastmag=&startabsmag=&starthostabs=&starthostcol=&startsavevis=&startlatevis=&startcurrvis=&startb=&startav=&endsavedate=&endpeakdate=&endlastdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=&endlastmag=&endabsmag=&endhostabs=&endhostcol=&endsavevis=&endlatevis=&endcurrvis=&endb=&endav=&sort=peakmag&format=csv",
#     "rcf_var":      "http://sites.astro.caltech.edu/ztf/rcf/explorer.php?f=s&coverage=any&samprcf=y&subsample=var&classstring=&classexclude=&refok=y&lcfig=y&ztflink=fritz&startsavedate=&startpeakdate=&startlastdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=&startlastmag=&startabsmag=&starthostabs=&starthostcol=&startsavevis=&startlatevis=&startcurrvis=&startb=&startav=&endsavedate=&endpeakdate=&endlastdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=&endlastmag=&endabsmag=&endhostabs=&endhostcol=&endsavevis=&endlatevis=&endcurrvis=&endb=&endav=&sort=peakmag&format=csv",
    
#     "rcf_deep_dim": "http://sites.astro.caltech.edu/ztf/rcf/explorer.php?f=s&coverage=any&sampdeep=y&subsample=all&classstring=&classexclude=&covok=y&refok=y&lcfig=y&ztflink=fritz&startsavedate=&startpeakdate=&startlastdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=18.5&startlastmag=&startabsmag=&starthostabs=&starthostcol=&startsavevis=&startlatevis=&startcurrvis=&startb=&startav=&endsavedate=&endpeakdate=&endlastdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=&endlastmag=&endabsmag=&endhostabs=&endhostcol=&endsavevis=&endlatevis=&endcurrvis=&endb=&endav=&sort=peakmag&format=csv",
#     "rcf_deep_var": "http://sites.astro.caltech.edu/ztf/rcf/explorer.php?f=s&coverage=any&sampdeep=y&subsample=var&classstring=&classexclude=&refok=y&lcfig=y&ztflink=fritz&startsavedate=&startpeakdate=&startlastdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=&startlastmag=&startabsmag=&starthostabs=&starthostcol=&startsavevis=&startlatevis=&startcurrvis=&startb=&startav=&endsavedate=&endpeakdate=&endlastdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=&endlastmag=&endabsmag=&endhostabs=&endhostcol=&endsavevis=&endlatevis=&endcurrvis=&endb=&endav=&sort=peakmag&format=csv"
# }

# # v3.2 (no changes to v4)
# query_urls = {
#     "trues": "http://sites.astro.caltech.edu/ztf/rcf/explorer.php?f=s&coverage=any&samprcf=y&sampdeep=y&subsample=trans&classstring=&classexclude=&refok=y&purity=y&ps1img=y&lcfig=y&ztflink=fritz&startsavedate=&startpeakdate=&startlastdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=&startlastmag=&startabsmag=&starthostabs=&starthostcol=&startsavevis=&startlatevis=&startcurrvis=&startb=&startav=&endsavedate=&endpeakdate=&endlastdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=18.5&endlastmag=&endabsmag=&endhostabs=&endhostcol=&endsavevis=&endlatevis=&endcurrvis=&endb=&endav=&sort=peakmag&format=csv",
#     "dims":  "http://sites.astro.caltech.edu/ztf/rcf/explorer.php?f=s&coverage=any&samprcf=y&sampdeep=y&subsample=all&classstring=&classexclude=&covok=y&refok=y&purity=y&lcfig=y&ztflink=fritz&startsavedate=&startpeakdate=&startlastdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=18.5&startlastmag=&startabsmag=&starthostabs=&starthostcol=&startsavevis=&startlatevis=&startcurrvis=&startb=&startav=&endsavedate=&endpeakdate=&endlastdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=&endlastmag=&endabsmag=&endhostabs=&endhostcol=&endsavevis=&endlatevis=&endcurrvis=&endb=&endav=&sort=peakmag&format=csv",
#     "vars":  "http://sites.astro.caltech.edu/ztf/rcf/explorer.php?f=s&coverage=any&samprcf=y&sampdeep=y&subsample=var&classstring=&classexclude=&refok=y&lcfig=y&ztflink=fritz&startsavedate=&startpeakdate=&startlastdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=&startlastmag=&startabsmag=&starthostabs=&starthostcol=&startsavevis=&startlatevis=&startcurrvis=&startb=&startav=&endsavedate=&endpeakdate=&endlastdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=&endlastmag=&endabsmag=&endhostabs=&endhostcol=&endsavevis=&endlatevis=&endcurrvis=&endb=&endav=&sort=peakmag&format=csv",
# }


In [4]:
# v4
query_urls = {
    "trues": "http://sites.astro.caltech.edu/ztf/rcf/explorer.php?f=s&coverage=any&samprcf=y&sampdeep=y&subsample=trans&classstring=&classexclude=&refok=y&purity=y&ps1img=y&lcfig=y&ztflink=fritz&startsavedate=&startpeakdate=&startlastdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=&startlastmag=&startabsmag=&starthostabs=&starthostcol=&startsavevis=&startlatevis=&startcurrvis=&startb=&startav=&endsavedate=&endpeakdate=&endlastdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=18.5&endlastmag=&endabsmag=&endhostabs=&endhostcol=&endsavevis=&endlatevis=&endcurrvis=&endb=&endav=&sort=peakmag&format=csv",
    "dims":  "http://sites.astro.caltech.edu/ztf/rcf/explorer.php?f=s&coverage=any&samprcf=y&sampdeep=y&subsample=all&classstring=&classexclude=&covok=y&refok=y&purity=y&lcfig=y&ztflink=fritz&startsavedate=&startpeakdate=&startlastdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=18.5&startlastmag=&startabsmag=&starthostabs=&starthostcol=&startsavevis=&startlatevis=&startcurrvis=&startb=&startav=&endsavedate=&endpeakdate=&endlastdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=&endlastmag=&endabsmag=&endhostabs=&endhostcol=&endsavevis=&endlatevis=&endcurrvis=&endb=&endav=&sort=peakmag&format=csv",
    "vars":  "http://sites.astro.caltech.edu/ztf/rcf/explorer.php?f=s&coverage=any&samprcf=y&sampdeep=y&subsample=var&classstring=&classexclude=&refok=y&lcfig=y&ztflink=fritz&startsavedate=&startpeakdate=&startlastdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=&startlastmag=&startabsmag=&starthostabs=&starthostcol=&startsavevis=&startlatevis=&startcurrvis=&startb=&startav=&endsavedate=&endpeakdate=&endlastdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=&endlastmag=&endabsmag=&endhostabs=&endhostcol=&endsavevis=&endlatevis=&endcurrvis=&endb=&endav=&sort=peakmag&format=csv",
}


In [5]:
# for set_name in query_urls.keys():
#     with open(f"data/base_data/{set_name}.csv", "w") as f:
#         f.write(requests.get(query_urls[set_name], auth=(creds["btsse_username"], creds["btsse_password"])).text)
#         print("Queried and wrote", set_name)


### Read queried data

In [6]:
for name1 in query_urls.keys():
    set1 = pd.read_csv(f"data/base_data/{name1}.csv")
    print(name1)
    for name2 in query_urls.keys():
        set2 = pd.read_csv(f"data/base_data/{name2}.csv")
        print(f"  in {name2}")
        print("  ", np.sum(set1['ZTFID'].isin(set2["ZTFID"])), "/", len(set1['ZTFID']))
    print()

trues
  in trues
   4476 / 4476
  in dims
   0 / 4476
  in vars
   0 / 4476

dims
  in trues
   0 / 7763
  in dims
   7763 / 7763
  in vars
   97 / 7763

vars
  in trues
   0 / 1083
  in dims
   97 / 1083
  in vars
   1083 / 1083



In [7]:
all_queries = pd.DataFrame(columns=["ZTFID"])
queries = [pd.read_csv(f"data/base_data/{set_name}.csv") for set_name in query_urls.keys()]

for i in range(len(queries)):
    queries[i] = queries[i][~queries[i]['ZTFID'].isin(all_queries['ZTFID'])]
    all_queries = pd.concat([all_queries, queries[i]])
    
trues = queries[0]
dims  = queries[1]
vars  = queries[2]

In [8]:
print(len(trues), "rcf true sources")
print(len(dims)+len(vars), "rcf (deep) false sources")

print(len(all_queries), "total")

# v2
# 7009 total

# v3.1
# 4150 rcf true sources
# 7913 rcf (deep) false sources
# 12063 total

# v3.2, v4
# 4476 rcf true sources
# 8749 rcf (deep) false sources
# 13225 total


4476 rcf true sources
8749 rcf (deep) false sources
13225 total


### Process list from Mat Smith

In [9]:
MS_Ias = pd.read_csv('mat-smith/ztfdr2_masterlist.csv')
MS_Ias.rename(columns={"ztfname": "ZTFID"}, inplace=True)
print("Total in MS list", len(MS_Ias))

Total in MS list 3793


In [10]:
nonZTF = ~MS_Ias['ZTFID'].str.contains('ZTF')
nonZTF_idxs = MS_Ias['ZTFID'].index[nonZTF]

MS_Ias = MS_Ias.drop(index=nonZTF_idxs)
print("Total in MS list excluding non ZTF objects", len(MS_Ias))

Total in MS list excluding non ZTF objects 3785


In [11]:
inBTSSE = MS_Ias['ZTFID'].isin(all_queries['ZTFID'])
inBTSSE_idxs = MS_Ias['ZTFID'].index[inBTSSE]

MS_Ias = MS_Ias.drop(index=inBTSSE_idxs)
print("New in MS list", len(MS_Ias))

New in MS list 1208


In [12]:
print("Total objects:", len(all_queries)+len(MS_Ias))

Total objects: 14433


### Objects to remove

In [13]:
objs_to_remove = ["ZTF18abdiasx", "ZTF21abyazip", "ZTF18aaadqua", "ZTF18aarrwmi"]
# print(len(all_queries)+len(MS_Ias))

for obj in objs_to_remove:
    trues = trues[trues["ZTFID"] != obj]
    dims = dims[dims["ZTFID"] != obj]
    vars = vars[vars["ZTFID"] != obj]
    MS_Ias = MS_Ias[MS_Ias["ZTFID"] != obj]
        
dims = dims[~dims["type"].isin(["bogus", "duplicate", "bogus?", "duplicate?"])]

queries = (trues, dims, vars) 
all_queries = pd.concat(queries)
all_ZTFIDs = np.concatenate((all_queries["ZTFID"].to_numpy(), MS_Ias['ZTFID'].to_numpy()))
print("Final number of objects:", len(all_ZTFIDs))


Final number of objects: 14353


### Helper functions for querying kowalski and processing alerts

In [14]:
def query_fritz(ZTFID, normalize=True, verbose=False):
    """
    Query fritz for alerts with cutouts for a (list of) ZTFID(s)

    Parameters
    ----------
    ZTFID: string or list
        Object IDs to query for (e.g. ZTF22abwqedu)
    
    normalize (optional): bool
        normalize cutouts by the Frobenius norm (L2) 
        
    Returns
    -------
    alerts: list of dicts
        each element in list is an alert
        alert keys include 

    ----------------------------------------------------------
    ADAPTED FROM https://github.com/growth-astro/ztfrest/
    https://zwickytransientfacility.github.io/ztf-avro-alert/schema.html
    """
    
    # Deal with provided input being a single ZTF object (string) and multiple (list)
    if type(ZTFID) == str:
        list_ZTFID = [ZTFID]
    elif type(ZTFID) == list:
        list_ZTFID = ZTFID
    else:
        print(f"{ZTFID} must be a list or a string")
        return None

    # Set up query details
    host = "https://fritz.science"
    metadata_endpoint = "alerts"
    triplets_endpoint = "alerts_triplets"
    headers = {'Authorization': f'token {api_token}'}
    
    alerts = []
    
    # For each object requested ...
    for ZTFID in list_ZTFID:
        # Set up metadata api endpoint url and execute request
        url = urllib.parse.urljoin(host, f'/api/{metadata_endpoint}/{ZTFID}')
        r = requests.get(url, headers=headers, params={})
        
        try:
            object_alerts = r.json()['data']
        except:
            # Query response does not have any data - failed query (connection or permissions)
            print(r.json())
        
        if object_alerts == []:
            # No alerts recieved - possibly because of permissions
            print("  No data for", ZTFID)
        
        if verbose:
            print(f"queried {len(object_alerts)} alerts of {ZTFID}")
            
        # some images will be corrupted, initialize array to log which to exclude
        to_drop = np.array((), dtype=int)
            
        # For each alert ...
        for i in range(len(object_alerts)):
            # candidate and classifications are two dicts nested within the alert dict
            # This merges those two dicts and does away with the unncessary data in the alert dict
            alert = object_alerts[i]['candidate'] | object_alerts[i]['classifications']
            alert["candid"] = object_alerts[i]['candid']
            
            # Set up triplets api endpoint url and execute request
            params={"candid": alert["candid"], "normalizeImage": normalize}
            url = urllib.parse.urljoin(host, f'/api/{triplets_endpoint}/{ZTFID}')
            r = requests.get(url, headers=headers, params=params)
            
            # Inset triplet into alert dict
            alert['triplet'] = np.asarray(r.json()['data']['triplet'])
            
            # Note the alert/triplet index where a cutout was found to be corrupted 
            if r.json()['data']['image_corrupt']:
                to_drop = np.append(to_drop, int(i))
                
            object_alerts[i] = alert
        
        # Delete corresponding triplets and alerts that had corrupted cutouts
        if len(to_drop) > 0:
            object_alerts = np.delete(object_alerts, list(to_drop), axis=0)
        
        alerts += list(object_alerts)
        
        if verbose:
            print(f"queried {len(object_alerts)} triplets of {ZTFID}")
            print("  Finished querying", ZTFID)
    if verbose:
        print(BOLD+f"Finished all queries, got {len(alerts)} alerts"+END+"\n")
    return alerts


In [59]:
def query_kowalski(ZTFID, kowalski, programid, normalize : bool = True, verbose : bool = False, save_raw = None, load_raw = None):
    """
    Query kowalski for alerts with cutouts for a (list of) ZTFID(s)

    Parameters
    ----------
    ZTFID: string or list
        Object IDs to query for (e.g. ZTF22abwqedu)
    
    kowalski:
        a kowalski api object created with the kowalski library
        
    normalize (optional): bool
        normalize cutouts by the Frobenius norm (L2)
        
    programid:
        which program to pull alerts from (1=public, 2=collab, 3=caltech mode)
        
    verbose (optional): bool
        print diagnostics after each query
        
    save_raw (optional): str
        if provided, all query results will be individually saved to disk at this path before any processsing is done
        
    load_raw (optional): str
        if provided, check for existing file at this path before querying, load file and continue processing as if just queried

    Returns
    -------
    alerts: list of dicts
        each dict represents alert
        alert columns include jd, ra, dec, candid, acai and braii scores, magpsf, cutouts, etc.
        
    
    ADAPTED FROM https://github.com/growth-astro/ztfrest/
    https://zwickytransientfacility.github.io/ztf-avro-alert/schema.html
    """
    
    # Deal with provided input being a single ZTF object (string) and multiple (list)
    if type(ZTFID) == str:
        list_ZTFID = [ZTFID]
    elif type(ZTFID) == list:
        list_ZTFID = ZTFID
    else:
        print(f"{ZTFID} must be a list or a string")
        return None

    alerts = []
    
    # For each object requested ...
    for ZTFID in list_ZTFID:
        # Set up query
        query = {
            "query_type": "find",
            "query": {
                "catalog": "ZTF_alerts",
                "filter": {
                    # take only alerts for specified object
                    'objectId': ZTFID,
                    # take only alerts with specified programid
                    "candidate.programid": programid,
                },
                # what quantities to recieve 
                "projection": {
                    "_id": 0,
                    "objectId": 1,
                    
                    "candidate.candid": 1,
                    "candidate.fid": 1,
                    "candidate.isdiffpos": 1,
                    "candidate.ndethist": 1,
                    "candidate.ncovhist": 1,
                    "candidate.sky": 1,
                    "candidate.fwhm": 1,
                    "candidate.seeratio": 1,
                    "candidate.mindtoedge": 1,
                    "candidate.nneg": 1,
                    "candidate.nbad": 1,
                    "candidate.scorr": 1,
                    "candidate.dsnrms": 1,
                    "candidate.ssnrms": 1,
                    "candidate.exptime": 1,
                    
                    "candidate.field": 1,
                    "candidate.jd": 1,
                    "candidate.ra": 1,
                    "candidate.dec": 1,
                    
                    "candidate.magpsf": 1,
                    "candidate.sigmapsf": 1,
                    "candidate.diffmaglim": 1,
                    "candidate.magap": 1,
                    "candidate.sigmagap": 1,
                    "candidate.magapbig": 1,
                    "candidate.sigmagapbig": 1,
                    "candidate.magdiff": 1,
                    "candidate.magzpsci": 1,
                    "candidate.magzpsciunc": 1,
                    "candidate.magzpscirms": 1,
                    
                    "candidate.distnr": 1,
                    "candidate.magnr": 1,
                    "candidate.sigmanr": 1,
                    "candidate.chinr": 1,
                    "candidate.sharpnr": 1,
                    
                    "candidate.neargaia": 1,
                    "candidate.neargaiabright": 1,
                    "candidate.maggaia": 1,
                    "candidate.maggaiabright": 1,    
                    
                    "candidate.drb": 1,
                    "candidate.classtar": 1,
                    "candidate.sgscore1": 1,
                    "candidate.distpsnr1": 1,
                    "candidate.sgscore2": 1,
                    "candidate.distpsnr2": 1,
                    "candidate.sgscore3": 1,
                    "candidate.distpsnr3": 1,
                                        
                    "classifications.acai_h": 1,
                    "classifications.acai_v": 1,
                    "classifications.acai_o": 1,
                    "classifications.acai_n": 1,
                    "classifications.acai_b": 1,
                    
                    "cutoutScience": 1,
                    "cutoutTemplate": 1,
                    "cutoutDifference": 1,
                }
            }
        }
        
        object_alerts = None
        existing_data_path = None
        
        # Check if file path is provided for locating preloaded data
        if type(load_raw) == str:
            existing_data_path = os.path.join(load_raw, f"{ZTFID}_prog{programid}.npy")
            
            if os.path.exists(existing_data_path):
                # Read existing data
                object_alerts = np.load(existing_data_path, allow_pickle=True)
                print(f"    loaded existing data for {ZTFID}")
            else:
                print(f"    could not find existing data for {ZTFID}")
                existing_data_path = None
        
        # if opting to not use preloaded data or preloaded data couldn't be found
        if object_alerts is None:
            # Execute query
            r = kowalski.query(query)
            
            if r['data'] == []:
                # No alerts recieved - possibly by failed query (connection or permissions)
                print(f"  No programid={programid} data for", ZTFID)
                continue
            else:
                # returned data is list of dicts, each dict is an alert packet
                object_alerts = r['data']   

        # Only try to save raw data if preloaded data couldn't be found
        if existing_data_path is None:
            if type(save_raw) == str:
                if not os.path.exists(save_raw):
                    os.makedirs(save_raw)
                np.save(os.path.join(save_raw, f"{ZTFID}_prog{programid}"), object_alerts)
            else:
                print(f"Could not find save directory: {save_raw}")
                print("No queries will be saved")
                save_raw = None

        # initialize empty array to contain triplets
        triplets = np.empty((len(object_alerts), 63, 63, 3))
        # some images will be corrupted, initialize array to log which to exclude
        to_drop = np.array((), dtype=int)

        # For each alert ...
        for i, alert in enumerate(object_alerts):
            # Unzip fits files of cutouts
            triplets[i], drop = make_triplet(alert, normalize=normalize)

            # Note the alert/triplet index where a cutout was found to be corrupted 
            if drop:
                to_drop = np.append(to_drop, int(i))

        # Delete corresponding triplets and alerts that had corrupted cutouts
        if len(to_drop) > 0:
            triplets = np.delete(triplets, list(to_drop), axis=0)
            object_alerts = np.delete(object_alerts, list(to_drop), axis=0)

        # candidate and classifications are two dicts nested within the alert dict
        # This merges those two dicts and does away with the unncessary data in the alert dict 
        # object_alerts = [alert['candidate'] | alert['classifications'] for alert in object_alerts]

        # Add triplet to the alert dict
        for alert, triplet in zip(object_alerts, triplets):
            alert['triplet'] = triplet

        alerts += list(object_alerts)

        if verbose:
            print(f"  Finished {'loading' if existing_data_path else 'querying'}", ZTFID)
    
    if verbose:
        print(f"Finished all programid={programid} queries, got {len(alerts)} alerts\n")
    
    return alerts


In [16]:
def make_triplet(alert, normalize: bool = True):
    """
    Unpack binary fits files containing cutouts from kowalski
    Helper function to query_kowalski()

    Parameters
    ----------
    alert: dict
        alert dictionary queried from kowlaski
        see query_kowalski()
    
    normalize (optional): bool
        normalize cutouts by the Frobenius norm (L2) 

    Returns
    -------
    triplet: 63 x 63 x 3 array
        3 channel 63 x 63 image representing the science, reference, and difference cutouts
    
    drop: bool
        whether or not the file is found to be corrupted
    
    ----------------------------------------------------------
    ADAPTED FROM https://github.com/dmitryduev/braai
    """
    
    cutout_dict = dict()
    drop = False
    
    for cutout in ('science', 'template', 'difference'):
        cutout_data = loads(dumps([alert[f'cutout{cutout.capitalize()}']['stampData']]))[0]
        # unzip fits file
        with gzip.open(io.BytesIO(cutout_data), 'rb') as f:
            with fits.open(io.BytesIO(f.read())) as hdu:
                data = hdu[0].data

                # Compute median value of image to fill nans
                medfill = np.nanmedian(data.flatten())
                
                # if the median is not a typical pixel value, image is corrupted; mark to be excluded
                if medfill == np.nan or medfill == -np.inf or medfill == np.inf:
                    print(alert['objectId'], "bad medfill (nan or inf)", alert['candidate']['candid'])
                    drop = True
                
                # Fill in nans with median value
                cutout_dict[cutout] = np.nan_to_num(data, nan=medfill)
                
                # normalize with L2 norm
                if normalize and not drop:
                    cutout_dict[cutout] /= np.linalg.norm(cutout_dict[cutout])
                    
                # If image is all zeros, image is corrupted; mark to be excluded
                if np.all(cutout_dict[cutout].flatten() == 0):
                    print(alert['objectId'], "zero image", alert['candidate']['candid'])
                    drop=True
                
                # If any nans remain in image, image is corrupted; mark to be excluded
                # Should never trigger because nans were already filled
#                 if np.any(np.isnan(cutout_dict[cutout].flatten())):
#                     print(alert['objectId'], "nan here", alert['candid'])
#                     drop=True
                    
        # pad to 63x63 if smaller
        shape = cutout_dict[cutout].shape
        if shape != (63, 63):
            print("bad shape", shape, alert['candidate']['candid'], alert['objectId'])
            # Fill value will have changed after normalizing so recompute
            medfill = np.nanmedian(cutout_dict[cutout].flatten())
            
            # Execute padding
            cutout_dict[cutout] = np.pad(cutout_dict[cutout],
                                         [(0, 63 - shape[0]),
                                          (0, 63 - shape[1])],
                                          mode='constant', 
                                          constant_values=medfill)
            
    triplet = np.zeros((63, 63, 3))
    triplet[:, :, 0] = cutout_dict['science']
    triplet[:, :, 1] = cutout_dict['template']
    triplet[:, :, 2] = cutout_dict['difference']
    
    return triplet, drop


In [17]:
def plot_triplet(tr, show_fig: bool = True):
    """ADAPTED FROM https://github.com/dmitryduev/braai"""
    fig = plt.figure(figsize=(8, 2), dpi=120)
    
    ax1 = fig.add_subplot(131)
    ax1.axis('off')
    ax1.imshow(tr[:, :, 0], origin='upper', cmap=plt.cm.bone, norm=LogNorm())
    ax1.title.set_text('Science')
    
    ax2 = fig.add_subplot(132)
    ax2.axis('off')
    ax2.imshow(tr[:, :, 1], origin='upper', cmap=plt.cm.bone, norm=LogNorm())
    ax2.title.set_text('Reference')
    
    ax3 = fig.add_subplot(133)
    ax3.axis('off')
    ax3.imshow(tr[:, :, 2], origin='upper', cmap=plt.cm.bone)
    ax3.title.set_text('Difference')

    if show_fig:
        plt.show()
    else:
        return fig
    

In [18]:
def extract_triplets(alerts, normalize: bool = True, pop_triplet: bool = True):
    """
    Takes in alerts (list of dicts) with key 'triplet', pops triplets out of alerts, 
    and returns alerts and triplets separated
    """
    triplets = np.empty((len(alerts), 63, 63, 3))
    for i, alert in enumerate(alerts):
        triplets[i] = alert['triplet']
        
        if pop_triplet:
            alert.pop('triplet'); alert.pop('cutoutScience'); alert.pop('cutoutTemplate'); alert.pop('cutoutDifference')
        
    return alerts, triplets


def prep_alerts(alerts, label):
    """
    takes in alerts (list of dicts) with nested dicts 'candidate' and 'classifications'
    un-nests inner dicts and adds column containing provided labels
    returns dataframe 
    """
    cand_class_data = [alert['candidate'] | alert['classifications'] for alert in alerts]

    df = pd.DataFrame(cand_class_data)
    df.insert(0, "objectId", [alert['objectId'] for alert in alerts])
#     df.insert(1, "candid", [alert['candid'] for alert in alerts])
    
    # label must be int equalling 0, 1 or a list of 1s and 0s
    if type(label) == list or type(label) == np.ndarray:
        assert(len(label) == len(alerts))
        df.insert(2, "label", label)
    elif type(label) == int:    
        df.insert(2, "label", np.full((len(alerts),), label, dtype=int))
    print("Arranged candidate data and inserted labels")
    return df


### Query data from kowalski, separate and save triplets and candidate data  

In [77]:
def download_training_data(source_df, set_name, kowalski, label, normalize_cutouts : bool = True, verbose : bool = False, save_raw = None, load_raw = None):
    """
    Downloads alerts with cutouts from kowalski
    Saves triplets in a .npy and alert metadata in a .csv
    
    Parameters
    ----------
    source_df: dataframe
        dataframe with columns "ZTFID"
    
    kowalski:
        a kowalski api object created with the penquins library
        
    label: int, array_like, or "compute"
        BTS / not BTS label to assign to each alert in saved csv
        if int (must be 0 or 1) assign all alerts provided label
        if array_like (length must match number of alerts) assign from array in order
        if "compute" assign all objects with any alert with magpsf < 18.5 label=1, otherwise 0
        
    normalize_cutouts (optional)- see query_kowalski()
        
    verbose (optional): bool
        print diagnostics
        
    save_raw, load_raw (optional) - see query_kowalski()

    Returns
    -------
    Nothing
    """
    
    if verbose:
        print(f"Querying kowalski for {len(source_df['ZTFID'])} objects of {set_name}")
        
    alerts, triplets = extract_triplets(query_kowalski(source_df['ZTFID'].to_list(), k, 1, normalize=normalize_cutouts, 
                                                       verbose=verbose, save_raw=save_raw, load_raw=load_raw) + 
                                        query_kowalski(source_df['ZTFID'].to_list(), k, 2, normalize=normalize_cutouts, 
                                                       verbose=verbose, save_raw=save_raw, load_raw=load_raw))

    np.save(f"data/base_data/{set_name}_triplets.npy", triplets)
    del triplets
    print("Saved and purged triplets\n")

    num_alerts = len(alerts)
    
    if type(label) == int:
        label = np.full((num_alerts), label, dtype=int)
    elif type(label) == list or type(label) == np.ndarray:
        label = label
    elif label == "compute":
        true_objs = set()
        for alert in alerts: 
            if alert['candidate']['magpsf'] < 18.5:
                true_objs.add(alert['objectId'])
        label = np.asarray([1 if alert['objectId'] in true_objs else 0 for alert in alerts])
    else:
        print(f"Could not understand label: {label}")
    
    num_trues = np.sum(label == 1)
    num_falses = np.sum(label == 0)
    if num_trues + num_falses != len(label):
        print(f"Invalid labels provided: {label}")
    else:
        print(f"{set_name} {len(label)} total alerts: {num_trues} trues, {num_falses} falses")

    cand_data = prep_alerts(alerts, label)
    cand_data.to_csv(f'data/base_data/{set_name}_candidates.csv', index=False)
    del cand_data
    print("Saved and purged candidate data")
    

In [20]:
download_training_data(trues, "trues", k, label=1, normalize_cutouts=True, verbose=True, save_raw=external_HDD+"trues", load_raw=external_HDD+"trues")


Querying kowalski for 4475 objects of trues
  Finished querying ZTF20abijfqq
  Finished querying ZTF21abiuvdk
  Finished querying ZTF21aaabvit
  Finished querying ZTF22aafoqrd
  Finished querying ZTF20aavpnlv
  Finished querying ZTF22abtknum
  Finished querying ZTF21aaqytjr
  Finished querying ZTF19aacgslb
  Finished querying ZTF20achlced
  Finished querying ZTF21abibafp
  Finished querying ZTF20aaelulu
  Finished querying ZTF18acbvgqw
  Finished querying ZTF21aaprfqv
ZTF18acbwaxk zero image 901177870315015009
  Finished querying ZTF18acbwaxk
  Finished querying ZTF19aatlmbo
bad shape (52, 63) 2130548985215015000 ZTF22absmrdt
bad shape (52, 63) 2130548985215015000 ZTF22absmrdt
bad shape (52, 63) 2130548985215015000 ZTF22absmrdt
bad shape (57, 63) 2133538875215015000 ZTF22absmrdt
bad shape (57, 63) 2133538875215015000 ZTF22absmrdt
bad shape (57, 63) 2133538875215015000 ZTF22absmrdt
bad shape (58, 63) 2133541275215015000 ZTF22absmrdt
bad shape (58, 63) 2133541275215015000 ZTF22absmrdt
ba

  Finished querying ZTF22abmygmc
  Finished querying ZTF22abqkzuq
  Finished querying ZTF20abeywdn
  Finished querying ZTF19aatesgp
  Finished querying ZTF21aaampui
  Finished querying ZTF22aajijjf
  Finished querying ZTF19aagmsrr
bad shape (63, 55) 1988318331415015000 ZTF22aanwibf
bad shape (63, 55) 1988318331415015000 ZTF22aanwibf
bad shape (63, 55) 1988318331415015000 ZTF22aanwibf
bad shape (63, 54) 1996208691415015002 ZTF22aanwibf
bad shape (63, 54) 1996208691415015002 ZTF22aanwibf
bad shape (63, 54) 1996208691415015002 ZTF22aanwibf
bad shape (63, 55) 1996251421415015001 ZTF22aanwibf
bad shape (63, 55) 1996251421415015001 ZTF22aanwibf
bad shape (63, 55) 1996251421415015001 ZTF22aanwibf
bad shape (63, 50) 2000235341415015001 ZTF22aanwibf
bad shape (63, 50) 2000235341415015001 ZTF22aanwibf
bad shape (63, 50) 2000235341415015001 ZTF22aanwibf
bad shape (63, 60) 2003233481415015001 ZTF22aanwibf
bad shape (63, 60) 2003233481415015001 ZTF22aanwibf
bad shape (63, 60) 2003233481415015001 ZT

  Finished querying ZTF22aaahewf
  Finished querying ZTF20aadzwnu
  Finished querying ZTF22abtdbnx
  Finished querying ZTF19aabyheu
  Finished querying ZTF19aavhxby
bad shape (62, 63) 828226686315015000 ZTF19aalvaxt
bad shape (62, 63) 828226686315015000 ZTF19aalvaxt
bad shape (62, 63) 828226686315015000 ZTF19aalvaxt
bad shape (62, 63) 839290496315015000 ZTF19aalvaxt
bad shape (62, 63) 839290496315015000 ZTF19aalvaxt
bad shape (62, 63) 839290496315015000 ZTF19aalvaxt
  Finished querying ZTF19aalvaxt
  Finished querying ZTF21achxlkq
  Finished querying ZTF22aajezaq
  Finished querying ZTF21abjbnhh
bad shape (63, 55) 2018351463415015004 ZTF22aasycgf
bad shape (63, 55) 2018351463415015004 ZTF22aasycgf
bad shape (63, 55) 2018351463415015004 ZTF22aasycgf
bad shape (63, 45) 2025291283415015004 ZTF22aasycgf
bad shape (63, 45) 2025291283415015004 ZTF22aasycgf
bad shape (63, 45) 2025291283415015004 ZTF22aasycgf
bad shape (63, 50) 2027419613415015000 ZTF22aasycgf
bad shape (63, 50) 20274196134150

  Finished querying ZTF21aaxuywf
  Finished querying ZTF19abdzehg
  Finished querying ZTF22abfvibh
  Finished querying ZTF18aawlxls
  Finished querying ZTF22aasxgjp
  Finished querying ZTF22aajkpen
  Finished querying ZTF21aalinof
  Finished querying ZTF18aczeesl
  Finished querying ZTF20abwrcmq
  Finished querying ZTF19aaynsyd
  Finished querying ZTF18abuqugw
  Finished querying ZTF19acfejbj
  Finished querying ZTF19abqysic
  Finished querying ZTF19aayfaum
ZTF19abqwtfu zero image 996172335815015011
ZTF19abqwtfu zero image 997169975815015015
ZTF19abqwtfu zero image 993200025815015024
ZTF19abqwtfu zero image 997198715815015014
ZTF19abqwtfu zero image 996201075815015017
ZTF19abqwtfu zero image 995172035815015018
ZTF19abqwtfu zero image 993171385815015015
ZTF19abqwtfu zero image 994170525815015022
ZTF19abqwtfu zero image 995200745815015018
ZTF19abqwtfu zero image 994198915815015014
ZTF19abqwtfu zero image 986171125815015007
ZTF19abqwtfu zero image 992196015815015020
ZTF19abqwtfu zero imag

  Finished querying ZTF18abzbkby
  Finished querying ZTF21acbcfmh
  Finished querying ZTF20aaurzce
  Finished querying ZTF20aanadlu
  Finished querying ZTF22abbkhja
  Finished querying ZTF20aaowuim
  Finished querying ZTF19acvrigb
  Finished querying ZTF18abgmcmv
  Finished querying ZTF21aahaksd
  Finished querying ZTF18aazgfkq
  Finished querying ZTF22aaoippk
  Finished querying ZTF20aazgtmp
bad shape (63, 45) 1730466460515015023 ZTF21aceepnp
bad shape (63, 45) 1730466460515015023 ZTF21aceepnp
bad shape (63, 45) 1730466460515015023 ZTF21aceepnp
bad shape (63, 52) 1735362300515015010 ZTF21aceepnp
bad shape (63, 52) 1735362300515015010 ZTF21aceepnp
bad shape (63, 52) 1735362300515015010 ZTF21aceepnp
bad shape (63, 46) 1735430020515015012 ZTF21aceepnp
bad shape (63, 46) 1735430020515015012 ZTF21aceepnp
bad shape (63, 46) 1735430020515015012 ZTF21aceepnp
bad shape (63, 58) 1742369120515015011 ZTF21aceepnp
bad shape (63, 58) 1742369120515015011 ZTF21aceepnp
bad shape (63, 58) 1742369120515

...

bad shape (63, 49) 951155044615015008 ZTF19aawgxdn
bad shape (63, 49) 951155044615015008 ZTF19aawgxdn
bad shape (63, 49) 951155044615015008 ZTF19aawgxdn
bad shape (63, 59) 946165714615015014 ZTF19aawgxdn
bad shape (63, 59) 946165714615015014 ZTF19aawgxdn
bad shape (63, 59) 946165714615015014 ZTF19aawgxdn
bad shape (63, 59) 950187704615015005 ZTF19aawgxdn
bad shape (63, 59) 950187704615015005 ZTF19aawgxdn
bad shape (63, 59) 950187704615015005 ZTF19aawgxdn
bad shape (63, 49) 943198074615015017 ZTF19aawgxdn
bad shape (63, 49) 943198074615015017 ZTF19aawgxdn
bad shape (63, 49) 943198074615015017 ZTF19aawgxdn
bad shape (63, 50) 949195844615015010 ZTF19aawgxdn
bad shape (63, 50) 949195844615015010 ZTF19aawgxdn
bad shape (63, 50) 949195844615015010 ZTF19aawgxdn
bad shape (63, 62) 978140974615015015 ZTF19aawgxdn
bad shape (63, 62) 978140974615015015 ZTF19aawgxdn
bad shape (63, 62) 978140974615015015 ZTF19aawgxdn
bad shape (63, 56) 973155744615015025 ZTF19aawgxdn
bad shape (63, 56) 973155744615

  Finished querying ZTF18abcsgvj
bad shape (61, 63) 1959161075015015000 ZTF22aahgnxz
bad shape (61, 63) 1959161075015015000 ZTF22aahgnxz
bad shape (61, 63) 1959161075015015000 ZTF22aahgnxz
bad shape (60, 63) 1959163435015015000 ZTF22aahgnxz
bad shape (60, 63) 1959163435015015000 ZTF22aahgnxz
bad shape (60, 63) 1959163435015015000 ZTF22aahgnxz
bad shape (52, 63) 1960154585015015000 ZTF22aahgnxz
bad shape (52, 63) 1960154585015015000 ZTF22aahgnxz
bad shape (52, 63) 1960154585015015000 ZTF22aahgnxz
bad shape (50, 63) 1960156945015015000 ZTF22aahgnxz
bad shape (50, 63) 1960156945015015000 ZTF22aahgnxz
bad shape (50, 63) 1960156945015015000 ZTF22aahgnxz
bad shape (49, 63) 1960159305015015000 ZTF22aahgnxz
bad shape (49, 63) 1960159305015015000 ZTF22aahgnxz
bad shape (49, 63) 1960159305015015000 ZTF22aahgnxz
bad shape (48, 63) 1960161675015015000 ZTF22aahgnxz
bad shape (48, 63) 1960161675015015000 ZTF22aahgnxz
bad shape (48, 63) 1960161675015015000 ZTF22aahgnxz
bad shape (47, 63) 196016405501

bad shape (63, 52) 624245486015015003 ZTF18abrzeym
bad shape (63, 52) 624245486015015003 ZTF18abrzeym
bad shape (63, 52) 607255726015015004 ZTF18abrzeym
bad shape (63, 52) 607255726015015004 ZTF18abrzeym
bad shape (63, 52) 607255726015015004 ZTF18abrzeym
bad shape (63, 53) 629128206015015003 ZTF18abrzeym
bad shape (63, 53) 629128206015015003 ZTF18abrzeym
bad shape (63, 53) 629128206015015003 ZTF18abrzeym
bad shape (63, 51) 630148836015015002 ZTF18abrzeym
bad shape (63, 51) 630148836015015002 ZTF18abrzeym
bad shape (63, 51) 630148836015015002 ZTF18abrzeym
bad shape (63, 62) 629150436015015004 ZTF18abrzeym
bad shape (63, 62) 629150436015015004 ZTF18abrzeym
bad shape (63, 62) 629150436015015004 ZTF18abrzeym
  Finished querying ZTF18abrzeym
  No programid=2 data for ZTF20aabconi
ZTF19aazudta zero image 936274966315015021
  Finished querying ZTF19aazudta
  No programid=2 data for ZTF18aaxjuwy
  Finished querying ZTF22aahhrks
  Finished querying ZTF20abexyxk
  No programid=2 data for ZTF21aa

  Finished querying ZTF20aazhtvi
  No programid=2 data for ZTF19acmwszk
  Finished querying ZTF21abzxkyv
  Finished querying ZTF19abqmjdq
  Finished querying ZTF19aaeoqst
  Finished querying ZTF20ackkejs
  No programid=2 data for ZTF21abaiicy
  Finished querying ZTF21abizpri
  Finished querying ZTF21aafkpnp
  Finished querying ZTF19aamwyzl
  No programid=2 data for ZTF19adcdrng
  No programid=2 data for ZTF20abpglei
bad shape (47, 63) 905174501715015000 ZTF19abaeyln
bad shape (47, 63) 905174501715015000 ZTF19abaeyln
bad shape (47, 63) 905174501715015000 ZTF19abaeyln
  Finished querying ZTF19abaeyln
bad shape (63, 60) 980208135415015001 ZTF19aayxlma
bad shape (63, 60) 980208135415015001 ZTF19aayxlma
bad shape (63, 60) 980208135415015001 ZTF19aayxlma
bad shape (63, 55) 980211955415015003 ZTF19aayxlma
bad shape (63, 55) 980211955415015003 ZTF19aayxlma
bad shape (63, 55) 980211955415015003 ZTF19aayxlma
bad shape (63, 56) 979207335415015002 ZTF19aayxlma
bad shape (63, 56) 979207335415015002

  Finished querying ZTF22aaegxpa
  No programid=2 data for ZTF20aauompx
  No programid=2 data for ZTF21aaqwbgq
  Finished querying ZTF21accyvil
  Finished querying ZTF19aanirdx
  No programid=2 data for ZTF21acizlcg
  Finished querying ZTF22aaafbma
  Finished querying ZTF21aaoiaar
  No programid=2 data for ZTF22abtdbdt
  Finished querying ZTF21aawiwqh
  No programid=2 data for ZTF20aamqslk
  Finished querying ZTF21abzithf
  Finished querying ZTF20acuhren
  No programid=2 data for ZTF20aanbfbf
  Finished querying ZTF19acokufi
  Finished querying ZTF21acjjxww
ZTF18abimsyv zero image 576181252915015052
ZTF18abimsyv zero image 576193252915015089
  Finished querying ZTF18abimsyv
  Finished querying ZTF18abnxhmc
  Finished querying ZTF19abgfuhh
  Finished querying ZTF22abfypms
  Finished querying ZTF21aagrjag
  Finished querying ZTF20abcsbux
  Finished querying ZTF22aaaowlo
  Finished querying ZTF18abdkimx
  No programid=2 data for ZTF20acowpur
  No programid=2 data for ZTF22aatrbvj
bad shap

In [67]:
download_training_data(dims, "dims", k, label=0, normalize_cutouts=True, verbose=True, save_raw=external_HDD+"dims", load_raw=to_desktop+"dims")


Querying kowalski for 7687 objects of dims
    loaded existing data for ZTF22abkunqa
  Finished loading ZTF22abkunqa
    loaded existing data for ZTF21abrxgiv
  Finished loading ZTF21abrxgiv
    loaded existing data for ZTF19adaylqy
  Finished loading ZTF19adaylqy
    loaded existing data for ZTF19acgjojy
  Finished loading ZTF19acgjojy
    loaded existing data for ZTF20acnywhz
  Finished loading ZTF20acnywhz
    loaded existing data for ZTF21aceccve
  Finished loading ZTF21aceccve
    loaded existing data for ZTF21abasxdp
  Finished loading ZTF21abasxdp
    loaded existing data for ZTF19aabyvbf
  Finished loading ZTF19aabyvbf
    loaded existing data for ZTF20acurrzb
  Finished loading ZTF20acurrzb
    loaded existing data for ZTF21abwprfg
  Finished loading ZTF21abwprfg
    loaded existing data for ZTF21aapliyn
  Finished loading ZTF21aapliyn
    loaded existing data for ZTF21aavodtn
  Finished loading ZTF21aavodtn
    loaded existing data for ZTF19aawudku
  Finished loading ZTF19aaw

  Finished loading ZTF22abqyiaa
    loaded existing data for ZTF20aaetirf
bad shape (63, 43) 1105216305215015006 ZTF20aaetirf
bad shape (63, 43) 1105216305215015006 ZTF20aaetirf
bad shape (63, 43) 1105216305215015006 ZTF20aaetirf
bad shape (63, 43) 1106154435215015009 ZTF20aaetirf
bad shape (63, 43) 1106154435215015009 ZTF20aaetirf
bad shape (63, 43) 1106154435215015009 ZTF20aaetirf
  Finished loading ZTF20aaetirf
    loaded existing data for ZTF19aauqwna
  Finished loading ZTF19aauqwna
    loaded existing data for ZTF20acoqlav
  Finished loading ZTF20acoqlav
    loaded existing data for ZTF19aatqzim
  Finished loading ZTF19aatqzim
    loaded existing data for ZTF21aawmtug
  Finished loading ZTF21aawmtug
    loaded existing data for ZTF21acjxzsa
  Finished loading ZTF21acjxzsa
    loaded existing data for ZTF21abvhxyk
  Finished loading ZTF21abvhxyk
    loaded existing data for ZTF19accobqx
  Finished loading ZTF19accobqx
    loaded existing data for ZTF22aaikall
  Finished loading ZTF

  Finished loading ZTF21aaqldsk
    loaded existing data for ZTF21aaxstsv
ZTF21aaxstsv zero image 1581315900615015052
ZTF21aaxstsv zero image 1584275590615015011
ZTF21aaxstsv zero image 1584316160615015012
ZTF21aaxstsv zero image 1586316230615015018
ZTF21aaxstsv zero image 1586352730615015011
ZTF21aaxstsv zero image 1588338240615015006
ZTF21aaxstsv zero image 1588399420615015015
ZTF21aaxstsv zero image 1590294840615015018
ZTF21aaxstsv zero image 1590380430615015009
ZTF21aaxstsv zero image 1592380400615015010
ZTF21aaxstsv zero image 1592357560615015006
ZTF21aaxstsv zero image 1594315300615015014
ZTF21aaxstsv zero image 1596274660615015012
ZTF21aaxstsv zero image 1600271230615015011
ZTF21aaxstsv zero image 1602215420615015035
ZTF21aaxstsv zero image 1608254860615015005
ZTF21aaxstsv zero image 1610286080615015009
ZTF21aaxstsv zero image 1610300900615015006
ZTF21aaxstsv zero image 1612226050615015006
ZTF21aaxstsv zero image 1616255240615015009
ZTF21aaxstsv zero image 1618307590615015012
  

  Finished loading ZTF21abxjcdc
    loaded existing data for ZTF19aamhmsx
  Finished loading ZTF19aamhmsx
    loaded existing data for ZTF19aachpob
  Finished loading ZTF19aachpob
    loaded existing data for ZTF21aaraqpr
  Finished loading ZTF21aaraqpr
    loaded existing data for ZTF21aaaouyd
  Finished loading ZTF21aaaouyd
    loaded existing data for ZTF19acbnhkd
  Finished loading ZTF19acbnhkd
    loaded existing data for ZTF19abzvyew
  Finished loading ZTF19abzvyew
    loaded existing data for ZTF19abgskjo
  Finished loading ZTF19abgskjo
    loaded existing data for ZTF20aclkcuu
  Finished loading ZTF20aclkcuu
    loaded existing data for ZTF22aaknhja
  Finished loading ZTF22aaknhja
    loaded existing data for ZTF21abpvrhn
  Finished loading ZTF21abpvrhn
    loaded existing data for ZTF21aciezlg
  Finished loading ZTF21aciezlg
    loaded existing data for ZTF19acszmgx
  Finished loading ZTF19acszmgx
    loaded existing data for ZTF22aauuxul
  Finished loading ZTF22aauuxul
    lo

  Finished loading ZTF18aaszvfn
    loaded existing data for ZTF21abotose
  Finished loading ZTF21abotose
    loaded existing data for ZTF21abzlozj
  Finished loading ZTF21abzlozj
    loaded existing data for ZTF21acjvgxg
  Finished loading ZTF21acjvgxg
    loaded existing data for ZTF19aadyoqz
  Finished loading ZTF19aadyoqz
    loaded existing data for ZTF18aaxmhvk
  Finished loading ZTF18aaxmhvk
    loaded existing data for ZTF22aapjrvz
  Finished loading ZTF22aapjrvz
    loaded existing data for ZTF21aaskyvp
  Finished loading ZTF21aaskyvp
    loaded existing data for ZTF20acovdho
  Finished loading ZTF20acovdho
    loaded existing data for ZTF21abqfomi
  Finished loading ZTF21abqfomi
    loaded existing data for ZTF21achplzp
  Finished loading ZTF21achplzp
    loaded existing data for ZTF20aayxdse
  Finished loading ZTF20aayxdse
    loaded existing data for ZTF19aanovps
  Finished loading ZTF19aanovps
    loaded existing data for ZTF21abqzays
  Finished loading ZTF21abqzays
    lo

...

  Finished querying ZTF21acjxpco
    could not find existing data for ZTF21abbftdu
  No programid=2 data for ZTF21abbftdu
    could not find existing data for ZTF22aahhexs
  No programid=2 data for ZTF22aahhexs
    could not find existing data for ZTF22aagxqya
  No programid=2 data for ZTF22aagxqya
    could not find existing data for ZTF22aauskml
  Finished querying ZTF22aauskml
    could not find existing data for ZTF22aakpzai
  No programid=2 data for ZTF22aakpzai
    could not find existing data for ZTF21abdgprl
  Finished querying ZTF21abdgprl
    could not find existing data for ZTF21acdwurx
  Finished querying ZTF21acdwurx
    could not find existing data for ZTF22aaorbox
  Finished querying ZTF22aaorbox
    could not find existing data for ZTF22abfyxyt
  No programid=2 data for ZTF22abfyxyt
    could not find existing data for ZTF21abctgsv
  Finished querying ZTF21abctgsv
    could not find existing data for ZTF21abdhdhm
  No programid=2 data for ZTF21abdhdhm
    could not find

  Finished querying ZTF22aarbswn
    could not find existing data for ZTF22aahhjuy
  Finished querying ZTF22aahhjuy
    could not find existing data for ZTF21abpoakl
  No programid=2 data for ZTF21abpoakl
    could not find existing data for ZTF22aaxpsru
  No programid=2 data for ZTF22aaxpsru
    could not find existing data for ZTF22aafgqjz
  No programid=2 data for ZTF22aafgqjz
    could not find existing data for ZTF19adeagka
  Finished querying ZTF19adeagka
    could not find existing data for ZTF21acjrscn
  No programid=2 data for ZTF21acjrscn
    could not find existing data for ZTF21abqkwqs
  No programid=2 data for ZTF21abqkwqs
    could not find existing data for ZTF21abcsaaf
  No programid=2 data for ZTF21abcsaaf
    could not find existing data for ZTF21abcomro
  No programid=2 data for ZTF21abcomro
    could not find existing data for ZTF22aaonpjx
  Finished querying ZTF22aaonpjx
    could not find existing data for ZTF21abspqgv
  No programid=2 data for ZTF21abspqgv
    co

  Finished querying ZTF22abiypsb
    could not find existing data for ZTF21abqezye
  No programid=2 data for ZTF21abqezye
    could not find existing data for ZTF21abeefpr
  No programid=2 data for ZTF21abeefpr
    could not find existing data for ZTF22aarmnxk
  Finished querying ZTF22aarmnxk
    could not find existing data for ZTF22aaiqyry
  No programid=2 data for ZTF22aaiqyry
    could not find existing data for ZTF21acqeebw
  No programid=2 data for ZTF21acqeebw
    could not find existing data for ZTF19aaddidt
  Finished querying ZTF19aaddidt
    could not find existing data for ZTF21abqtdxz
  Finished querying ZTF21abqtdxz
    could not find existing data for ZTF22aasinsf
bad shape (57, 63) 2062293833815015000 ZTF22aasinsf
bad shape (57, 63) 2062293833815015000 ZTF22aasinsf
bad shape (57, 63) 2062293833815015000 ZTF22aasinsf
  Finished querying ZTF22aasinsf
    could not find existing data for ZTF21aavcdiq
  No programid=2 data for ZTF21aavcdiq
    could not find existing data f

  No programid=2 data for ZTF21aaygkho
    could not find existing data for ZTF21abfxqfi
  Finished querying ZTF21abfxqfi
    could not find existing data for ZTF21aaxtzbm
  Finished querying ZTF21aaxtzbm
    could not find existing data for ZTF22aboeraw
  Finished querying ZTF22aboeraw
    could not find existing data for ZTF22aamsrfj
  No programid=2 data for ZTF22aamsrfj
    could not find existing data for ZTF21abjgzhn
  No programid=2 data for ZTF21abjgzhn
    could not find existing data for ZTF22aauknta
  Finished querying ZTF22aauknta
    could not find existing data for ZTF21aaxwnxu
  No programid=2 data for ZTF21aaxwnxu
    could not find existing data for ZTF22abnyvqr
  Finished querying ZTF22abnyvqr
    could not find existing data for ZTF21abjqiah
  No programid=2 data for ZTF21abjqiah
    could not find existing data for ZTF21abyquvq
  No programid=2 data for ZTF21abyquvq
    could not find existing data for ZTF21abfzdkr
  No programid=2 data for ZTF21abfzdkr
    could no

  Finished querying ZTF21abvmblm
    could not find existing data for ZTF22abfxfzw
  Finished querying ZTF22abfxfzw
    could not find existing data for ZTF22aazpcfy
  No programid=2 data for ZTF22aazpcfy
    could not find existing data for ZTF21abxgiwm
  No programid=2 data for ZTF21abxgiwm
    could not find existing data for ZTF20acreckb
  No programid=2 data for ZTF20acreckb
    could not find existing data for ZTF21aavxgus
  No programid=2 data for ZTF21aavxgus
    could not find existing data for ZTF21abaxofw
  No programid=2 data for ZTF21abaxofw
    could not find existing data for ZTF18aakybym
  Finished querying ZTF18aakybym
    could not find existing data for ZTF19aadavfl
  Finished querying ZTF19aadavfl
    could not find existing data for ZTF22aazxuda
  No programid=2 data for ZTF22aazxuda
    could not find existing data for ZTF21abvyzxg
  No programid=2 data for ZTF21abvyzxg
    could not find existing data for ZTF22aamrlrd
  No programid=2 data for ZTF22aamrlrd
    co

In [22]:
download_training_data(vars, "vars", k, label=0, normalize_cutouts=True, verbose=True, save_raw=external_HDD+"vars", load_raw=external_HDD+"vars")


Querying kowalski for 984 objects of vars
  Finished querying ZTF20actzmzp
  Finished querying ZTF21ablazyz
  Finished querying ZTF20acyxwzf
  Finished querying ZTF20aabbimu
  Finished querying ZTF21aanehlz
  Finished querying ZTF19aaxugwk
  Finished querying ZTF20acioekr
  Finished querying ZTF19abxvksn
  Finished querying ZTF20aakficj
  Finished querying ZTF19aaywash
  Finished querying ZTF22aajjqti
  Finished querying ZTF20acgyjry
  Finished querying ZTF20acjpwas
  Finished querying ZTF19abzipjc
  Finished querying ZTF20abcdrsm
  Finished querying ZTF20acqucpz
  Finished querying ZTF19aakiygv
  Finished querying ZTF20aagqrbx
  Finished querying ZTF21aagoldn
  Finished querying ZTF21abntqyx
bad shape (63, 57) 1021432802615010005 ZTF17aaaocpq
bad shape (63, 57) 1021432802615010005 ZTF17aaaocpq
bad shape (63, 57) 1021432802615010005 ZTF17aaaocpq
bad shape (63, 50) 1036349932615015017 ZTF17aaaocpq
bad shape (63, 50) 1036349932615015017 ZTF17aaaocpq
bad shape (63, 50) 1036349932615015017

  Finished querying ZTF21abcnlpj
  Finished querying ZTF20abrubfr
  Finished querying ZTF21abmemav
  Finished querying ZTF19aabimbg
  Finished querying ZTF19abzyxgz
  Finished querying ZTF18acuxibg
  Finished querying ZTF20ablyczu
  Finished querying ZTF20abwurfg
  Finished querying ZTF20aczacou
  Finished querying ZTF20abzcokk
  Finished querying ZTF18aaheszr
  Finished querying ZTF19aaodojg
  Finished querying ZTF20acufmah
  Finished querying ZTF19abxzumq
  Finished querying ZTF18adbcqwv
  Finished querying ZTF21aaadtpw
  Finished querying ZTF19aavajub
  Finished querying ZTF18abrlrma
  Finished querying ZTF19adbqznr
  Finished querying ZTF21aairadl
  Finished querying ZTF19aaumtxz
  Finished querying ZTF19abbwmem
  Finished querying ZTF19acqdbad
  Finished querying ZTF20abjirmc
  Finished querying ZTF18abkojhi
  Finished querying ZTF19abasxmh
  Finished querying ZTF21achzkjo
  Finished querying ZTF18abxmbfq
  Finished querying ZTF19abcoosk
  Finished querying ZTF18acwycwi
  Finished

  Finished querying ZTF20aapcjjo
  Finished querying ZTF20abhfrxt
  Finished querying ZTF18abnyktn
bad shape (61, 63) 2099163562615015000 ZTF22abgcigh
bad shape (61, 63) 2099163562615015000 ZTF22abgcigh
bad shape (61, 63) 2099163562615015000 ZTF22abgcigh
  Finished querying ZTF22abgcigh
  Finished querying ZTF18acxwtbe
  Finished querying ZTF18abmbdbg
  Finished querying ZTF20aavnpug
  Finished querying ZTF19achbhfc
  Finished querying ZTF20abazbfr
  Finished querying ZTF20aahklzo
  Finished querying ZTF20aawbodq
  Finished querying ZTF20abehusj
  Finished querying ZTF18abvarea
  Finished querying ZTF19abupyxe
  Finished querying ZTF20aawzrlj
  Finished querying ZTF19aargomy
  Finished querying ZTF19abogdfr
  Finished querying ZTF19aanhffu
  Finished querying ZTF18aahfohe
  Finished querying ZTF19abzwbxy
  Finished querying ZTF21abotogu
  Finished querying ZTF22aakzsrj
  Finished querying ZTF21aatscez
  Finished querying ZTF22aakhqrd
  Finished querying ZTF18abmxatl
  Finished querying

  Finished querying ZTF21abgjdtt
  Finished querying ZTF21aagnbev
  Finished querying ZTF22aaefmbp
  Finished querying ZTF19aabyoce
  Finished querying ZTF20abojvde
  Finished querying ZTF20actowqe
  Finished querying ZTF20aakugak
  Finished querying ZTF20aaxckge
  Finished querying ZTF18ablookq
  Finished querying ZTF20abkhjbu
  Finished querying ZTF19abhbiqq
  Finished querying ZTF19aaflcgn
bad shape (63, 45) 1369173770215015010 ZTF18abxcues
bad shape (63, 45) 1369173770215015010 ZTF18abxcues
bad shape (63, 45) 1369173770215015010 ZTF18abxcues
bad shape (63, 45) 1658381730215015049 ZTF18abxcues
bad shape (63, 45) 1658381730215015049 ZTF18abxcues
bad shape (63, 45) 1658381730215015049 ZTF18abxcues
bad shape (63, 54) 1733220160215015006 ZTF18abxcues
bad shape (63, 54) 1733220160215015006 ZTF18abxcues
bad shape (63, 54) 1733220160215015006 ZTF18abxcues
bad shape (63, 60) 1788115870215015013 ZTF18abxcues
bad shape (63, 60) 1788115870215015013 ZTF18abxcues
bad shape (63, 60) 1788115870215

  Finished querying ZTF19acpmpvh
  Finished querying ZTF20aazvump
bad shape (52, 63) 880419971715015000 ZTF19aaweqnm
bad shape (52, 63) 880419971715015000 ZTF19aaweqnm
bad shape (52, 63) 880419971715015000 ZTF19aaweqnm
bad shape (48, 63) 880459461715015000 ZTF19aaweqnm
bad shape (48, 63) 880459461715015000 ZTF19aaweqnm
bad shape (48, 63) 880459461715015000 ZTF19aaweqnm
bad shape (62, 63) 883459021715015000 ZTF19aaweqnm
bad shape (62, 63) 883459021715015000 ZTF19aaweqnm
bad shape (62, 63) 883459021715015000 ZTF19aaweqnm
bad shape (50, 63) 889438981715015000 ZTF19aaweqnm
bad shape (50, 63) 889438981715015000 ZTF19aaweqnm
bad shape (50, 63) 889438981715015000 ZTF19aaweqnm
bad shape (58, 63) 895481421715015000 ZTF19aaweqnm
bad shape (58, 63) 895481421715015000 ZTF19aaweqnm
bad shape (58, 63) 895481421715015000 ZTF19aaweqnm
bad shape (62, 63) 914299571715015000 ZTF19aaweqnm
bad shape (62, 63) 914299571715015000 ZTF19aaweqnm
bad shape (62, 63) 914299571715015000 ZTF19aaweqnm
bad shape (53, 6

...

bad shape (47, 63) 1227279545715015000 ZTF18aalsiji
bad shape (47, 63) 1227279545715015000 ZTF18aalsiji
bad shape (47, 63) 1227279545715015000 ZTF18aalsiji
bad shape (47, 63) 1231279015715015000 ZTF18aalsiji
bad shape (47, 63) 1231279015715015000 ZTF18aalsiji
bad shape (47, 63) 1231279015715015000 ZTF18aalsiji
bad shape (46, 63) 1231319425715015000 ZTF18aalsiji
bad shape (46, 63) 1231319425715015000 ZTF18aalsiji
bad shape (46, 63) 1231319425715015000 ZTF18aalsiji
bad shape (41, 63) 1231339565715015000 ZTF18aalsiji
bad shape (41, 63) 1231339565715015000 ZTF18aalsiji
bad shape (41, 63) 1231339565715015000 ZTF18aalsiji
bad shape (50, 63) 1232282625715015000 ZTF18aalsiji
bad shape (50, 63) 1232282625715015000 ZTF18aalsiji
bad shape (50, 63) 1232282625715015000 ZTF18aalsiji
bad shape (49, 63) 1232303205715015000 ZTF18aalsiji
bad shape (49, 63) 1232303205715015000 ZTF18aalsiji
bad shape (49, 63) 1232303205715015000 ZTF18aalsiji
bad shape (54, 63) 1232320785715015000 ZTF18aalsiji
bad shape (5

  Finished querying ZTF20abpbfzp
  Finished querying ZTF20acutkgr
  Finished querying ZTF18aahvcby
  Finished querying ZTF19aaynnae
bad shape (49, 63) 899344620615015000 ZTF18abrzevn
bad shape (49, 63) 899344620615015000 ZTF18abrzevn
bad shape (49, 63) 899344620615015000 ZTF18abrzevn
bad shape (49, 63) 900328630615015000 ZTF18abrzevn
bad shape (49, 63) 900328630615015000 ZTF18abrzevn
bad shape (49, 63) 900328630615015000 ZTF18abrzevn
  Finished querying ZTF18abrzevn
  No programid=2 data for ZTF18acbwkqu
  No programid=2 data for ZTF19acjiett
  Finished querying ZTF19abxrdlu
  No programid=2 data for ZTF20acpguww
  Finished querying ZTF19aasltzs
  Finished querying ZTF21aatnpnl
  Finished querying ZTF18acjnusv
  Finished querying ZTF19aacipwt
  No programid=2 data for ZTF19abgknfb
  Finished querying ZTF19acbwrsu
  Finished querying ZTF19acevlfc
  Finished querying ZTF18abrnttt
  No programid=2 data for ZTF18acvvslx
  Finished querying ZTF21aaomiwf
  Finished querying ZTF18aamyexb
  Fi

  Finished querying ZTF18aafjeeu
bad shape (55, 63) 1751128553415015005 ZTF19aawscqd
bad shape (55, 63) 1751128553415015005 ZTF19aawscqd
bad shape (55, 63) 1751128553415015005 ZTF19aawscqd
bad shape (55, 63) 1784110163415015007 ZTF19aawscqd
bad shape (55, 63) 1784110163415015007 ZTF19aawscqd
bad shape (55, 63) 1784110163415015007 ZTF19aawscqd
bad shape (58, 63) 1788128253415015007 ZTF19aawscqd
bad shape (58, 63) 1788128253415015007 ZTF19aawscqd
bad shape (58, 63) 1788128253415015007 ZTF19aawscqd
bad shape (60, 63) 2148226893415015011 ZTF19aawscqd
bad shape (60, 63) 2148226893415015011 ZTF19aawscqd
bad shape (60, 63) 2148226893415015011 ZTF19aawscqd
  Finished querying ZTF19aawscqd
  Finished querying ZTF21abbxnbm
  Finished querying ZTF18ablusuz
  Finished querying ZTF18aadnqwz
bad shape (63, 50) 846300624915015008 ZTF18acvgnil
bad shape (63, 50) 846300624915015008 ZTF18acvgnil
bad shape (63, 50) 846300624915015008 ZTF18acvgnil
  Finished querying ZTF18acvgnil
  No programid=2 data for

  No programid=2 data for ZTF21acatkss
  Finished querying ZTF18adjkwyk
  Finished querying ZTF20acodsyb
  Finished querying ZTF18abedgvo
  Finished querying ZTF22aachgve
  Finished querying ZTF18aasxahh
  Finished querying ZTF20abjyzhj
  Finished querying ZTF19aazdroc
  Finished querying ZTF21abugfqj
  No programid=2 data for ZTF18ablsaka
  Finished querying ZTF18abubmjo
  Finished querying ZTF19aaikxsg
  No programid=2 data for ZTF19abljehr
  No programid=2 data for ZTF20aaxxhhi
  Finished querying ZTF20abeokzg
  Finished querying ZTF18aaddhpb
  Finished querying ZTF19accczao
  Finished querying ZTF18adallcx
  No programid=2 data for ZTF19aakptvj
  Finished querying ZTF19acdytef
  Finished querying ZTF20aadzfzy
  Finished querying ZTF18abwugyc
  Finished querying ZTF18adlmpkc
  Finished querying ZTF20acufmrl
  Finished querying ZTF20acqislg
  Finished querying ZTF18aaxtmeg
  Finished querying ZTF19abbxvlp
  No programid=2 data for ZTF20acwyoqx
  Finished querying ZTF18acxhvzb
  Finis

  Finished querying ZTF18acehiav
  No programid=2 data for ZTF21aagqijr
  Finished querying ZTF18accdnfn
  Finished querying ZTF19abanmqt
  No programid=2 data for ZTF20ablkoym
  No programid=2 data for ZTF18adopgzt
  Finished querying ZTF20abvywzu
  Finished querying ZTF19aabzfqe
  Finished querying ZTF19aaskowq
  No programid=2 data for ZTF20abdafxp
  Finished querying ZTF20acmvvkg
  Finished querying ZTF21acihfqr
  Finished querying ZTF22abdapsq
bad shape (62, 63) 1515506994715015001 ZTF21aamxlbf
bad shape (62, 63) 1515506994715015001 ZTF21aamxlbf
bad shape (62, 63) 1515506994715015001 ZTF21aamxlbf
  Finished querying ZTF21aamxlbf
  Finished querying ZTF22abgucbp
  No programid=2 data for ZTF22aalmjfs
  Finished querying ZTF22aaaflnh
Finished all programid=2 queries, got 45589 alerts

Saved and purged triplets

vars: 0 trues, 115606 falses
Arranged candidate data and inserted labels
Saved and purged candidate data


In [81]:
download_training_data(MS_Ias, "MS", k, label='compute', normalize_cutouts=True, verbose=True, save_raw=external_HDD+"MS")


Querying kowalski for 1207 objects of MS
  Finished querying ZTF18aaanzph
  Finished querying ZTF18aaaoaeq
  Finished querying ZTF18aaapivw
  Finished querying ZTF18aaasvcd
ZTF18aabasml zero image 464210840615010000
ZTF18aabasml zero image 470202170615010000
ZTF18aabasml zero image 479186670615010000
ZTF18aabasml zero image 491166660615010000
bad shape (63, 53) 494239500615010001 ZTF18aabasml
bad shape (63, 53) 494239500615010001 ZTF18aabasml
bad shape (63, 53) 494239500615010001 ZTF18aabasml
bad shape (63, 62) 500185470615010001 ZTF18aabasml
ZTF18aabasml zero image 500185470615010001
bad shape (63, 62) 500185470615010001 ZTF18aabasml
bad shape (63, 62) 500185470615010001 ZTF18aabasml
bad shape (63, 52) 503226220615010000 ZTF18aabasml
bad shape (63, 52) 503226220615010000 ZTF18aabasml
bad shape (63, 52) 503226220615010000 ZTF18aabasml
bad shape (63, 59) 506184860615010000 ZTF18aabasml
ZTF18aabasml zero image 506184860615010000
bad shape (63, 59) 506184860615010000 ZTF18aabasml
bad shap

  Finished querying ZTF18aabvpeg
  Finished querying ZTF18aabxrjp
bad shape (63, 52) 1226228400515010000 ZTF18aabyhlc
bad shape (63, 52) 1226228400515010000 ZTF18aabyhlc
bad shape (63, 52) 1226228400515010000 ZTF18aabyhlc
bad shape (63, 48) 1243225370515010000 ZTF18aabyhlc
bad shape (63, 48) 1243225370515010000 ZTF18aabyhlc
bad shape (63, 48) 1243225370515010000 ZTF18aabyhlc
bad shape (63, 60) 1243283040515010001 ZTF18aabyhlc
bad shape (63, 60) 1243283040515010001 ZTF18aabyhlc
bad shape (63, 60) 1243283040515010001 ZTF18aabyhlc
bad shape (63, 61) 1283196970515010000 ZTF18aabyhlc
bad shape (63, 61) 1283196970515010000 ZTF18aabyhlc
bad shape (63, 61) 1283196970515010000 ZTF18aabyhlc
bad shape (63, 59) 1290198920515010000 ZTF18aabyhlc
bad shape (63, 59) 1290198920515010000 ZTF18aabyhlc
bad shape (63, 59) 1290198920515010000 ZTF18aabyhlc
bad shape (63, 52) 1475538480515010003 ZTF18aabyhlc
bad shape (63, 52) 1475538480515010003 ZTF18aabyhlc
bad shape (63, 52) 1475538480515010003 ZTF18aabyhl

  Finished querying ZTF18aaccuih
  Finished querying ZTF18aaemivw
bad shape (51, 63) 608159152215010001 ZTF18aagrcfl
bad shape (51, 63) 608159152215010001 ZTF18aagrcfl
bad shape (51, 63) 608159152215010001 ZTF18aagrcfl
bad shape (47, 63) 611155082215010000 ZTF18aagrcfl
bad shape (47, 63) 611155082215010000 ZTF18aagrcfl
bad shape (47, 63) 611155082215010000 ZTF18aagrcfl
bad shape (57, 63) 615130342215010001 ZTF18aagrcfl
bad shape (57, 63) 615130342215010001 ZTF18aagrcfl
bad shape (57, 63) 615130342215010001 ZTF18aagrcfl
bad shape (62, 63) 618133862215010001 ZTF18aagrcfl
bad shape (62, 63) 618133862215010001 ZTF18aagrcfl
bad shape (62, 63) 618133862215010001 ZTF18aagrcfl
bad shape (59, 63) 862280682215010001 ZTF18aagrcfl
bad shape (59, 63) 862280682215010001 ZTF18aagrcfl
bad shape (59, 63) 862280682215010001 ZTF18aagrcfl
bad shape (57, 63) 852456882215010000 ZTF18aagrcfl
bad shape (57, 63) 852456882215010000 ZTF18aagrcfl
bad shape (57, 63) 852456882215010000 ZTF18aagrcfl
bad shape (50, 6

  Finished querying ZTF18aagrcfl
  Finished querying ZTF18aagrcir
  Finished querying ZTF18aagrebu
bad shape (63, 58) 2003254812915010004 ZTF18aagrgao
bad shape (63, 58) 2003254812915010004 ZTF18aagrgao
bad shape (63, 58) 2003254812915010004 ZTF18aagrgao
  Finished querying ZTF18aagrgao
  Finished querying ZTF18aagrtxs
  Finished querying ZTF18aagstdc
  Finished querying ZTF18aagtcxj
  Finished querying ZTF18aaguhgb
  Finished querying ZTF18aahesrp
  Finished querying ZTF18aahfeiy
bad shape (44, 63) 460216482215015000 ZTF18aahfgsk
bad shape (44, 63) 460216482215015000 ZTF18aahfgsk
bad shape (44, 63) 460216482215015000 ZTF18aahfgsk
bad shape (49, 63) 467246772215015000 ZTF18aahfgsk
bad shape (49, 63) 467246772215015000 ZTF18aahfgsk
bad shape (49, 63) 467246772215015000 ZTF18aahfgsk
bad shape (57, 63) 491174782215015000 ZTF18aahfgsk
bad shape (57, 63) 491174782215015000 ZTF18aahfgsk
bad shape (57, 63) 491174782215015000 ZTF18aahfgsk
bad shape (44, 63) 710506082215010000 ZTF18aahfgsk
bad 

bad shape (61, 63) 1887323325815010000 ZTF18aairuad
bad shape (51, 63) 1917369745815010000 ZTF18aairuad
bad shape (51, 63) 1917369745815010000 ZTF18aairuad
bad shape (51, 63) 1917369745815010000 ZTF18aairuad
bad shape (45, 63) 1917430605815010000 ZTF18aairuad
bad shape (45, 63) 1917430605815010000 ZTF18aairuad
bad shape (45, 63) 1917430605815010000 ZTF18aairuad
bad shape (60, 63) 1920415035815010000 ZTF18aairuad
bad shape (60, 63) 1920415035815010000 ZTF18aairuad
bad shape (60, 63) 1920415035815010000 ZTF18aairuad
bad shape (50, 63) 1924347075815010000 ZTF18aairuad
bad shape (50, 63) 1924347075815010000 ZTF18aairuad
bad shape (50, 63) 1924347075815010000 ZTF18aairuad
bad shape (56, 63) 1930284055815010000 ZTF18aairuad
bad shape (56, 63) 1930284055815010000 ZTF18aairuad
bad shape (56, 63) 1930284055815010000 ZTF18aairuad
bad shape (47, 63) 1947365945815010000 ZTF18aairuad
bad shape (47, 63) 1947365945815010000 ZTF18aairuad
bad shape (47, 63) 1947365945815010000 ZTF18aairuad
bad shape (5

...

  Finished querying ZTF18acrdciw
  No programid=2 data for ZTF18acrdlrp
  Finished querying ZTF18acrdwcf
  Finished querying ZTF18acrewzd
  No programid=2 data for ZTF18acrfklz
  Finished querying ZTF18acrfond
  Finished querying ZTF18acrgydk
  Finished querying ZTF18acrhbuk
  No programid=2 data for ZTF18acrtykk
  Finished querying ZTF18acrukkl
  No programid=2 data for ZTF18acrvakb
  No programid=2 data for ZTF18acrxpxd
  Finished querying ZTF18acryfsy
  Finished querying ZTF18acsblap
  Finished querying ZTF18acslhcq
  No programid=2 data for ZTF18acslknd
  Finished querying ZTF18acsolgq
  No programid=2 data for ZTF18acsouxk
  Finished querying ZTF18acsovsw
  Finished querying ZTF18acsoyiu
  No programid=2 data for ZTF18acswrpr
  Finished querying ZTF18acsxvrp
  No programid=2 data for ZTF18acsycdi
  No programid=2 data for ZTF18acsycnc
  Finished querying ZTF18acsynfy
  No programid=2 data for ZTF18acszgnm
  No programid=2 data for ZTF18actacux
  Finished querying ZTF18actasgb
  No

  Finished querying ZTF19aaripqw
  Finished querying ZTF19aarnrbw
  Finished querying ZTF19aasckwd
  Finished querying ZTF19aascpkb
  Finished querying ZTF19aasigfv
  Finished querying ZTF19aaskscv
  Finished querying ZTF19aaslzjf
  Finished querying ZTF19aasmftm
  Finished querying ZTF19aatgsdl
  No programid=2 data for ZTF19aatnefp
  No programid=2 data for ZTF19aatnfci
  No programid=2 data for ZTF19aatrbqs
  No programid=2 data for ZTF19aatvlxl
  No programid=2 data for ZTF19aatvlym
  Finished querying ZTF19aatvmbo
  No programid=2 data for ZTF19aatzlmw
  No programid=2 data for ZTF19aauhbua
  Finished querying ZTF19aauvczw
  Finished querying ZTF19aauvkxu
  No programid=2 data for ZTF19aauxepf
  No programid=2 data for ZTF19aavbkqk
  Finished querying ZTF19aavmnpb
  No programid=2 data for ZTF19aavnwoh
  No programid=2 data for ZTF19aavnwzv
  No programid=2 data for ZTF19aavorpa
  Finished querying ZTF19aavowpa
  No programid=2 data for ZTF19aavyafo
  Finished querying ZTF19aawfbz

  No programid=2 data for ZTF19acazpyz
  Finished querying ZTF19acbjqcf
  No programid=2 data for ZTF19acblhgq
  Finished querying ZTF19acblpaq
  Finished querying ZTF19acblqjf
  Finished querying ZTF19acbsuhn
  Finished querying ZTF19acbtcyx
  No programid=2 data for ZTF19acbtkmt
  Finished querying ZTF19acbtlkz
  Finished querying ZTF19acbuujf
  Finished querying ZTF19acbvllm
  Finished querying ZTF19acbvrdu
  No programid=2 data for ZTF19acbvrli
  Finished querying ZTF19accibkt
  Finished querying ZTF19acclupd
  Finished querying ZTF19acctjbv
  Finished querying ZTF19acdfkqa
  Finished querying ZTF19acdlaer
  Finished querying ZTF19aceeexa
  No programid=2 data for ZTF19acenjem
  No programid=2 data for ZTF19acfduxa
  Finished querying ZTF19acfsazg
  Finished querying ZTF19acftfst
  No programid=2 data for ZTF19acfvuxk
  Finished querying ZTF19acfxbxe
  No programid=2 data for ZTF19acgldqr
  Finished querying ZTF19acgnets
  Finished querying ZTF19acgpnlk
  Finished querying ZTF19ach

  Finished querying ZTF20aavofki
  Finished querying ZTF20aavxqej
  Finished querying ZTF20aavxraf
  Finished querying ZTF20aavynba
  Finished querying ZTF20aawmkxv
  Finished querying ZTF20aawnwxo
  No programid=2 data for ZTF20aaxbvtz
  No programid=2 data for ZTF20aaxcbvg
  Finished querying ZTF20aaxlnle
  Finished querying ZTF20aaybqae
  Finished querying ZTF20aayxcyn
  Finished querying ZTF20aayxkqy
  No programid=2 data for ZTF20aazhzen
  Finished querying ZTF20aaznlnj
  No programid=2 data for ZTF20aaznoxg
  Finished querying ZTF20aazomcu
  Finished querying ZTF20aazpnxl
  Finished querying ZTF20aazppkz
  Finished querying ZTF20aazqiel
  No programid=2 data for ZTF20ababkom
  Finished querying ZTF20abasewu
  Finished querying ZTF20abbarut
  Finished querying ZTF20abbbsfs
bad shape (63, 44) 1242285340815015023 ZTF20abbilvk
bad shape (63, 44) 1242285340815015023 ZTF20abbilvk
bad shape (63, 44) 1242285340815015023 ZTF20abbilvk
bad shape (63, 60) 1243320420815015015 ZTF20abbilvk
bad

  Finished querying ZTF20abqmmhn
  No programid=2 data for ZTF20abqqwui
  Finished querying ZTF20abqsrfe
  Finished querying ZTF20absitlr
bad shape (63, 58) 1325403965215015023 ZTF20abstsxb
bad shape (63, 58) 1325403965215015023 ZTF20abstsxb
bad shape (63, 58) 1325403965215015023 ZTF20abstsxb
bad shape (63, 48) 1331412605215015026 ZTF20abstsxb
bad shape (63, 48) 1331412605215015026 ZTF20abstsxb
bad shape (63, 48) 1331412605215015026 ZTF20abstsxb
bad shape (63, 57) 1332369745215015029 ZTF20abstsxb
bad shape (63, 57) 1332369745215015029 ZTF20abstsxb
bad shape (63, 57) 1332369745215015029 ZTF20abstsxb
bad shape (63, 59) 1333379815215015014 ZTF20abstsxb
bad shape (63, 59) 1333379815215015014 ZTF20abstsxb
bad shape (63, 59) 1333379815215015014 ZTF20abstsxb
bad shape (63, 48) 1359304785215015021 ZTF20abstsxb
bad shape (63, 48) 1359304785215015021 ZTF20abstsxb
bad shape (63, 48) 1359304785215015021 ZTF20abstsxb
bad shape (63, 48) 1361328285215015033 ZTF20abstsxb
bad shape (63, 48) 13613282852

### Visualization helper functions

In [ ]:
def plot_lightcurve(alerts):
    fid_to_color = {
        1: ('green',  'g'),
        2: ('red',    'r'),
        3: ('orange', 'i')
    }
    
    alerts = alerts.sort_values(by='jd')
    jds = alerts['jd']-alerts['jd'].to_numpy()[0]
        
    fig, ax = plt.subplots(figsize=(8,5))
    
    for fid in [1, 2, 3]:
        obs_in_filt = alerts['fid'] == fid
        
        alerts_in_filt = alerts.loc[obs_in_filt]
        plt.errorbar(jds[obs_in_filt], alerts_in_filt['magpsf'], fmt='o', color=fid_to_color[fid][0], yerr=alerts_in_filt['sigmapsf'], label='ztf'+fid_to_color[fid][1])
    
    ax.invert_yaxis()
    ax.set_xlabel("days since first detection", size=16, labelpad=10)
    ax.set_ylabel("PSF magnitude", size=16, labelpad=10)
    ax.legend(loc='upper right', bbox_to_anchor=(1.25, 1))
    
    return fig, ax

In [ ]:
def plot_lightcurve_itr(alerts, triplets):
    fid_to_color = {
        1: ('green',  'g'),
        2: ('red',    'r'),
        3: ('orange', 'i')
    }
    
#     alerts = alerts.sort_values(by='jd')
    first_detect = np.min(alerts['jd'].to_numpy())
    print(first_detect)
    for trip in trips[::10]:
        
        plot_triplet(trip, show_fig=True)
        
    fig, ax = plt.subplots(figsize=(8,5))
    
    for idx in alerts.index:
        alert = alerts.loc[idx]
        jd = alert['jd'] - first_detect
        if alert['isdiffpos']:
            plt.errorbar(jd, alert['magpsf'], fmt='^', fillstyle='none', alpha=0.5, color=fid_to_color[alert['fid']][0], yerr=alert['sigmapsf'])
        else:
            plt.errorbar(jd, alert['magpsf'], fmt='v', fillstyle='none', alpha=0.5, color=fid_to_color[alert['fid']][0], yerr=alert['sigmapsf'])
    
    
    plt.plot([],[], marker='^', color='red', fillstyle='none', alpha=0.5, label='ztfr pos diff')
    plt.plot([],[], marker='^', color='green', fillstyle='none', alpha=0.5, label='ztfg pos diff')
    plt.plot([],[], marker='^', color='orange', fillstyle='none', alpha=0.5, label='ztfi pos diff')
    
    plt.plot([],[], marker='v', color='r', fillstyle='none', alpha=0.5, label='ztfr neg diff')
    plt.plot([],[], marker='v', color='green', fillstyle='none', alpha=0.5, label='ztfg neg diff')
    plt.plot([],[], marker='v', color='orange', fillstyle='none', alpha=0.5, label='ztfi neg diff')
    ax.invert_yaxis()
    ax.set_xlabel("days since first detection", size=16, labelpad=10)
    ax.set_ylabel("PSF magnitude", size=16, labelpad=10)
    ax.legend(loc='upper right')#, bbox_to_anchor=(1.2,1))
    
    return fig, ax

In [ ]:
# ztfid = "ZTF18ablmduj"

# alerts = cand[cand['objectId']==ztfid]
# trips = triplets[cand['objectId']==ztfid]
# fig, ax = plot_lightcurve_itr(alerts, trips)
# ax.set_title(f"{ztfid} lightcurve", size=14)
# fig.tight_layout()
# plt.show()